In [1]:
%load_ext autoreload
%autoreload 2 
%reload_ext autoreload
%matplotlib inline
import matplotlib.pyplot as plt

# always import gbm_algos first !
import xgboost, lightgbm, catboost
from gplearn.genetic import SymbolicRegressor

# To access the contents of the parent dir
import sys; sys.path.insert(0, '../')
import os
from scipy.io import loadmat
from utils import *
from preprocess import *

# Let's do facy optimizers
from optimizers import Lookahead, AdamGC, SGDGC
from madgrad import MADGRAD
from lbfgsnew import LBFGSNew

# Modify at /usr/local/lib/python3.9/site-packages/torch_lr_finder/lr_finder.py
from torch_lr_finder import LRFinder

# Tracking
from tqdm import trange

import sympy
import sympytorch

# Finite difference method
from findiff import FinDiff, coefficients, Coefficient

In [2]:
include_N_res = False

DATA_PATH = '../PDE_FIND_experimental_datasets/kdv.mat'
X, T, Exact = space_time_grid(data_path=DATA_PATH, real_solution=True)
X_star, u_star = get_trainable_data(X, T, Exact)

# Doman bounds
lb = X_star.min(axis=0)
ub = X_star.max(axis=0)

N = 2000
print(f"Fine-tuning with {N} samples")
idx = np.random.choice(X_star.shape[0], N, replace=False)
X_u_train = X_star[idx, :]
u_train = u_star[idx,:]

# Unsup data
if include_N_res:
    N_res = N//2
    idx_res = np.array(range(X_star.shape[0]-1))[~idx]
    idx_res = np.random.choice(idx_res.shape[0], N_res, replace=True)
    X_res = X_star[idx_res, :]
    print(f"Fine-tuning with {N_res} unsup samples")
    X_u_train = np.vstack([X_u_train, X_res])
    u_train = np.vstack([u_train, torch.rand(X_res.shape[0], 1) - 1000])
    # del X_res
else: print("Not including N_res")

# Convert to torch.tensor
X_u_train = to_tensor(X_u_train, True)
u_train = to_tensor(u_train, False)
X_star = to_tensor(X_star, True)
u_star = to_tensor(u_star, False)

# lb and ub are used in adversarial training
scaling_factor = 1.0
lb = scaling_factor*to_tensor(lb, False)
ub = scaling_factor*to_tensor(ub, False)

# Feature names, base on the symbolic regression results
feature_names=('uf', 'u_x', 'u_xxx'); feature2index = {}

Fine-tuning with 2000 samples
Not including N_res


In [3]:
fdc = FinDiffCalculator(X, T, Exact)

dx = 0.1171875
dt = 0.1


In [4]:
u_t = fdc.finite_diff(1, diff_order=1)
u_t

array([[ 1.05854861e-04],
       [ 1.19884958e-04],
       [ 1.34786835e-04],
       ...,
       [-2.70758458e-06],
       [-1.38436125e-05],
       [-5.05822994e-05]])

In [5]:
derivatives = fdc.finite_diff_from_feature_names(('uf', 'u_x', 'u_xx', 'u_xxx', 'x'))
derivatives

{'uf': array([[9.07916333e-05],
        [1.02078624e-04],
        [1.14768625e-04],
        ...,
        [2.03811113e-05],
        [2.08336844e-05],
        [1.76123888e-05]]),
 'u_x': array([[-2.78722879e-04],
        [-2.69713352e-04],
        [-2.25364121e-04],
        ...,
        [ 6.99618177e-05],
        [ 3.80613650e-05],
        [ 8.29801061e-05]]),
 'u_xx': array([[0.0024192 ],
        [0.00176008],
        [0.00068528],
        ...,
        [0.00246367],
        [0.00054935],
        [0.00206211]]),
 'u_xxx': array([[-0.0274856 ],
        [ 0.00688773],
        [ 0.00164758],
        ...,
        [ 0.05990887],
        [ 0.0091197 ],
        [ 0.04635517]]),
 'x': array([[-30.       ],
        [-29.8828125],
        [-29.765625 ],
        ...,
        [ 29.6484375],
        [ 29.765625 ],
        [ 29.8828125]])}